Inference for EfficientNetB0 trained on 224x224 resizings

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from tqdm.notebook import tqdm_notebook
tqdm_notebook.pandas()
from zipfile import ZipFile
from io import StringIO
import cv2 as cv
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
np.random.seed(42)
import tensorflow as tf
from tensorflow import keras
import os

In [ ]:
filenames = os.listdir('../input/ranzcr-clip-catheter-line-classification/test/')
study_instances = [filename[:-4] for filename in filenames]

In [ ]:
def get_and_resize_all_images(X):
  all_img = []
  for image_name in tqdm_notebook(X):
    img = get_image(image_name)
    all_img.append(cv.resize(img,(224,224)))
  return np.asarray(all_img)

In [ ]:
def get_image(image_name):
  return mpimg.imread('../input/ranzcr-clip-catheter-line-classification/test/'+image_name+'.jpg')

In [ ]:
X_test_images = get_and_resize_all_images(study_instances)

In [ ]:
def three_channel_images(X):
  X2 = X.reshape(X.shape + (1,))
  return np.concatenate([X2,X2,X2],axis=3)

In [ ]:
X_test_images_three_channel = three_channel_images(X_test_images)

In [ ]:
X_test_images_three_channel.shape

In [ ]:
efficientnetb0 = keras.models.load_model('../input/efficientnetb0-21/efficientnetb0_2')

In [ ]:
efficientnetb0.compile(loss='binary_crossentropy',
              optimizer=tf.optimizers.Nadam(learning_rate=1e-05),
              metrics=[keras.metrics.AUC(multi_label=True,name='averaged_auc')])

In [ ]:
predictions = efficientnetb0.predict(X_test_images_three_channel)

In [ ]:
df_sub = pd.read_csv('../input/ranzcr-clip-catheter-line-classification/sample_submission.csv')

In [ ]:
df_sub['StudyInstanceUID'] = study_instances

In [ ]:
df_sub.iloc[:, 1:] = predictions

In [ ]:
df_sub.to_csv('submission.csv',index=False)